# Ch7. 유동성 모델링

- 유동성 위험의 원인: 완전 시장과 대칭적 정보 패러다임으로부터의 이탈 -> 도덕적 해이와 역선택으로 이어질 수 있음
- 유동성을 고려한 모델: 유동성이 요구되는 자산 수익과 불확실성 수준 모두에 영향을 미침 ~ more applicable
- 유동성: 채무불이행 가능성 추정에 있어 상당히 중요함
- 유동성 자산: 상당한 가격 영향 없이 대량의 자산이 판매될 수 있는 상태 ~ 거래 비용이라고도 함

(\*) 유동성을 정의하는 네 가지 특성
- 집중도: 자산을 같은 가격에 동시에 거래할 수 있는 능력 ~ 거래 비용 높으면 매수와 매도 가격의 차이 큼
- 즉시성: 대량의 매수 or 매도 주문을 거래할 수 있는 속도
- 깊이: 다양한 가격으로 풍부한 주문을 처리할 수 있는 많은 수의 구매자와 판매자가 존재하는지
- 탄력성: 불균형에서 회복하는 시장의 능력 ~ 주문 불균형이 빠르게 해소되는 가격 회복 과정

- 군집화 분석 -> 유동성 측정 군집화 -> 투자자가 유동성의 어느 부분에 집중해야 하는지 이해
- 군집화 분석: by 가우스 혼합 모델(GMM), 가우스 혼합 코풀라 모델(GMCM)
- GMM: 타원 분포에 잘 작동
- GMCM: 상관관계 고려 ~ GMM의 확장

(\*) 유동성 척도: 거래량 / 거래 비용 / 가격 영향 / 시장 영향

1. 거래량 기반 유동성 척도
- 시장이 깊을 때, 즉 금융 시장이 풍부한 주문을 충족할 수 있는 능력이 있을 때 대규모 주문이 처리됨
- 시장에 깊이가 없으면, 주문 불균형과 불연속성이 나타남

(i) 유동성 비율: 1%의 가격 변동을 유발하는 데 필요한 거래량의 정도 측정
$\\ LR_{it}=\frac{\sum^T_{t=1}P_{it}V_{it}}{\sum^T_{t=1}|PC_{it}}$
- $P_{it}$: t일의 주식 i의 총 가격
- $V_{it}$: t일의 주식 i의 거래량
- $|PC_{it}|$: t일과 t-1 에서의 가격 차이의 절댓값
- 비율이 높을수록 자산 i의 유동성이 높아짐 ~ 높은 거래량 / 낮은 가격차

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize']=(10,6)
pd.set_option('use_inf_as_na', True)

In [2]:
liq_data=pd.read_csv('bid_ask.csv')
liq_data.head()

,Unnamed: 0,Date,EXCHCD,TICKER,COMNAM,BIDLO,ASKHI,PRC,VOL,RET,SHROUT,OPENPRC,vwretx
0,1031570,2019-01-02,3.0,INTC,INTEL CORP,45.77,47.470,47.08,18761673.0,0.003196,4564000.0,45.960,0.001783
1,1031571,2019-01-03,3.0,INTC,INTEL CORP,44.39,46.280,44.49,32254097.0,-0.055013,4564000.0,46.150,-0.021219
2,1031572,2019-01-04,3.0,INTC,INTEL CORP,45.54,47.570,47.22,35419836.0,0.061362,4564000.0,45.835,0.033399
3,1031573,2019-01-07,3.0,INTC,INTEL CORP,46.75,47.995,47.44,22724997.0,0.004659,4564000.0,47.100,0.009191
4,1031574,2019-01-08,3.0,INTC,INTEL CORP,46.78,48.030,47.74,22721240.0,0.006324,4564000.0,47.800,0.010240


In [3]:
# 롤링 윈도우
rolling_five=[]
for j in liq_data.TICKER.unique():
    for i in range(len(liq_data[liq_data.TICKER==j])):
        rolling_five.append(liq_data[i:i+5].agg({'BIDLO':'min', 'ASKHI':'max','VOL':'sum',
                                                 'SHROUT':'mean','PRC':'mean'})) # function to use for aggregating data
rolling_five_df=pd.DataFrame(rolling_five)
rolling_five_df.columns=['bidlo_min','askhi_max','vol_sum','shrout_mean','prc_mean']
liq_vol_all=pd.concat([liq_data, rolling_five_df], axis=1)


In [4]:
liq_ratio=[]
for j in liq_vol_all.TICKER.unique():
    for i in range(len(liq_vol_all[liq_vol_all.TICKER==j])):
        liq_ratio.append((liq_vol_all['PRC'][i+1:i+6]*liq_vol_all['VOL'][i+1:i+6]).sum()
                         /(np.abs(liq_vol_all['PRC'][i+1:i+6].mean()-liq_vol_all['PRC'][i:i+5].mean())))

(ii) 휘-회벨 비율
$\\ L_{HH}=\frac{P_{max}-P_{min}}{P_{min}}/V/\bar{P}\times\text{shrout}$
- $P_{max},\ P_{min}$: 각 정해진 기간 동안의 최고/최저가
- $\bar{P}$: 정해진 기간 동안의 평균 종가

In [5]:
Lhh=[]
for j in liq_vol_all.TICKER.unique():
    for i in range(len(liq_vol_all[liq_vol_all.TICKER==j])):
        Lhh.append((liq_vol_all['PRC'][i:i+5].max()-liq_vol_all['PRC'][i:i+5].min())/
                   liq_vol_all['PRC'][i:i+5].min()/
                   (liq_vol_all['VOL'][i:i+5].sum()/liq_vol_all['SHROUT'][i:i+5].mean()*liq_vol_all['PRC'][i:i+5].mean()))

(iii) 회전율: 발행 주식 수에 대한 변동성의 비율
$LR_{it}=\frac{1}{D_{it}}\frac{\sum^T_{t=1}Vol_{it}}{\sum^T_{t=1}\text{shrout}_{it}}$
- $D_{it}$: 거래일 수 
- $Vol_{it}$: 시간 t에 거래된 주식 수
- $\text{shrout}_{it}$: 시간 t에 발행된 주식 수
- 회전율: 일일 데이터 기반 ~ 거래 빈도를 의미함

In [6]:
turnover_ratio=[]
for j in liq_vol_all.TICKER.unique():
    for i in range(len(liq_vol_all[liq_vol_all.TICKER==j])):
        turnover_ratio.append((1/liq_vol_all['VOL'].count())*
                              (np.sum(liq_vol_all['VOL'][i:i+1])/np.sum(liq_vol_all['SHROUT'][i:i+1])))
liq_vol_all['liq_ratio']=pd.DataFrame(liq_ratio)
liq_vol_all['Lhh']=pd.DataFrame(Lhh)
liq_vol_all['turnover_ratio']=pd.DataFrame(turnover_ratio)

2. 거래 비용 기반 유동성 측정
- 거래 비용: 투자자가 거래 중에 부담해야 하는 비용 ~ 거래의 집행과 관련된 모든 비용
- 명시적 비용: 주문 처리, 세금 및 중개 수수료와 관련
- 암묵적 비용: 매수-매도 스프레드, 실행 시기 등과 같은 더 많은 잠재 비용을 포함함
- 높은 거래 비용 -> 투자자의 거래 방해 -> 시장의 구매자와 판매자 수 줄임 -> 거래 장소가 더 단편화된 시장으로 분기돼 얕은 시장 형성을 초래함
- 거래 비용 저렴 -> 시장이 더욱 중앙 집중화 됨
- 즉시성: 유동성의 다른 차원 ~ 거래 비용과 밀접한 관련
- 매수-매도 스프레드가 거래 비용 잘 분석 -> 자산을 현금으로 전환하는 용이성을 결정할 수 있는 유동성의 좋은 지표임

(\*) 명목 비율과 유효 매수-매도 스프레드
1. 명목 스프레드: 거래 완료 비용, 즉 매수-매도 스프레드의 차이 측정
$\\ = \frac{P_{ask}-P_{bid}}{P_{mid}}$
2. 유효 스프레드: 거래 가격과 중간 가격 사이의 편차 측정
$\\ \frac{2|P_t-P_{mid}|}{P_{mid}}$

In [7]:
liq_vol_all['mid_price']=(liq_vol_all.ASKHI+liq_vol_all.BIDLO)/2
liq_vol_all['percent_quoted_ba']=(liq_vol_all.ASKHI-liq_vol_all.BIDLO)/liq_vol_all.mid_price
liq_vol_all['percent_effective_ba']=2*abs(
    (liq_vol_all.PRC-liq_vol_all.mid_price)
)/liq_vol_all.mid_price

(\*) 롤의 스프레드 추정
$\\ = \sqrt{-cov(\Delta P_t,\Delta P_{t-1})}$

In [ ]:
liq_vol_all['price_diff']=liq_vol_all.groupby('TICKER')['PRC'].diff()
liq_vol_all.dropna(inplace=True)
roll=[]

for j in liq_vol_all.TICKER.unique():
    for i in range(len(liq_vol_all[liq_vol_all.TICKER==j])):
        roll_cov=np.cov(liq_vol_all['price_diff'][i:i+5],liq_vol_all['price_diff'][i+1:i+6]) # 공분산 행렬 반환
        if roll_cov[0,1] < 0:
            roll.append(2*np.sqrt(-roll_cov[0,1]))
        else:
            roll.append(2*np.sqrt(np.abs(roll_cov[0,1])))

(\*) 코윈-슐츠 스프레드
- 하루의 고-저가 비율은 주식의 분산과 매수 호가 스프레드를 모두 반영함

In [16]:
gamma=[]
for j in liq_vol_all.TICKER.unique():
    for i in range(len(liq_vol_all[liq_vol_all.TICKER==j])):
        gamma.append(
            (max(liq_vol_all['ASKHI'].iloc[i+1], liq_vol_all['ASKHI'].iloc[i])
             -min(liq_vol_all['BIDLO'].iloc[i+1], liq_vol_all['BIDLO'].iloc[i]))**2
        )
gamma_array=np.array(gamma)

In [18]:
beta=[]
for j in liq_vol_all.TICKER.unique():
    for i in range(len(liq_vol_all[liq_vol_all.TICKER==j])):
        beta.append(
            (liq_vol_all['ASKHI'].iloc[i+1]-liq_vol_all['BIDLO'].iloc[i+1])**2 +
            (liq_vol_all['ASKHI'].iloc[i]-liq_vol_all['BIDLO'].iloc[i])**2
        )
beta_array=np.array(beta)

In [20]:
alpha=((np.sqrt(2*beta_array)-np.sqrt(beta_array))/
       (3-(2*np.sqrt(2))))-np.sqrt(gamma_array/(3-(2*np.sqrt(2))))
CS_spread=(2*(np.exp(alpha)-1))/(1+np.exp(alpha))

In [21]:
liq_vol_all=liq_vol_all.reset_index()
liq_vol_all['roll']=pd.DataFrame(roll)
liq_vol_all['CS_spread']=pd.DataFrame(CS_spread)